In [1]:
"""
이더리움 트레이딩 백테스팅 시스템 (Data Leakage 방지 버전)
29개 모델 × 7개 Fold × 8개 포지션 전략 × 3가지 거래 방식
"""

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

UPSTAGE_API_KEY = "up_VVMdYIYXa3F6qY8ns8fO8kRGNSeWQ"

# ==================== 설정 ====================
@dataclass
class Config:
    base_path: Path = Path("../model_results/2025-10-26")
    raw_data_path: Path = Path("../macro_data/macro_data/macro_crypto_data.csv") 
    output_path: Path = None
    
    models: List[str] = field(default_factory=lambda: 
                              ['RandomForest', 'LightGBM', 'XGBoost', 'SVM', 'LogisticRegression', 'NaiveBayes', 
                               'KNN', 'AdaBoost', 'CatBoost', 'DecisionTree', 'ExtraTrees', 'Bagging', 
                               'GradientBoosting', 'HistGradientBoosting', 'StackingEnsemble', 'VotingHard', 
                               'VotingSoft', 'MLP', 'LSTM', 'BiLSTM', 'GRU', 'TCN', 'CNN_LSTM', 
                               'LSTM_Attention', 'DTW_LSTM', 'VMD_Hybrid', 'EMD_LSTM', 'Hybrid_LSTM_GRU', 
                               'Residual_LSTM'])
    
    folds: List[int] = field(default_factory=lambda: list(range(1, 8)))
    
    position_strategies: Dict[str, float] = field(default_factory=lambda: {
        'fixed_01': 0.01, 'fixed_05': 0.05, 'fixed_10': 0.10,
        'fixed_30': 0.30, 'fixed_50': 0.50, 'fixed_80': 0.80,
        'kelly': None, 'confidence_based': None
    })
    
    initial_capital: float = 1_000_000  # 100만원
    trading_cost: float = 0.004  # 0.4%
    
    # Swing Trading 파라미터
    swing_trend_window: int = 5
    swing_entry_threshold: float = 0.35
    swing_exit_threshold: float = 0.15
    swing_min_confidence: float = 0.3
    swing_stop_loss: float = -0.03
    swing_take_profit: float = 0.15
    swing_max_holding: int = 20
    
    # Kelly Criterion 최소 거래 수
    kelly_min_trades: int = 20
    
    def __post_init__(self):
        if self.output_path is None:
            self.output_path = self.base_path / "backtest_results_comprehensive"
        self.output_path.mkdir(parents=True, exist_ok=True)
        (self.output_path / "detailed_trades").mkdir(exist_ok=True)

# ==================== 개선된 데이터 로더 ====================
class ImprovedDataLoader:
    """백테스팅용 데이터 로더 - 원본 가격 병합"""
    
    def __init__(self, config: Config):
        self.config = config
        
        # ✅ 원본 데이터 로드 (한 번만)
        print(f"Loading raw price data from: {config.raw_data_path}")
        self.raw_prices = pd.read_csv(config.raw_data_path)
        self.raw_prices['date'] = pd.to_datetime(self.raw_prices['date'])
        
        # 백테스팅에 필요한 컬럼만 선택
        price_cols = ['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close', 'ETH_Volume']
        available_cols = [col for col in price_cols if col in self.raw_prices.columns]
        
        if len(available_cols) < 5:
            raise ValueError(f"원본 데이터에 필수 컬럼 누락: {set(price_cols) - set(available_cols)}")
        
        self.raw_prices = self.raw_prices[available_cols].copy()
        print(f"✓ Loaded {len(self.raw_prices)} rows of price data")
    
    def load_fold_data(self, fold: int) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame]]:
        """Fold별 시장 데이터와 모델 예측 로드 + 원본 가격 병합"""
        fold_name = f"fold_{fold}_walk_forward_rolling_reverse"
        
        # ===== 1. 예측/타겟 데이터 로드 =====
        market_path = self.config.base_path / "raw_data/direction/walk_forward" / fold_name / "test_raw.csv"
        
        if not market_path.exists():
            raise FileNotFoundError(f"Market data not found: {market_path}")
        
        market_df = pd.read_csv(market_path)
        market_df['date'] = pd.to_datetime(market_df['date'])
        
        # Target 컬럼
        target_cols = ['date', 'next_open', 'next_close', 'next_direction', 'next_log_return']
        available_targets = [col for col in target_cols if col in market_df.columns]
        market_df = market_df[available_targets].copy()
        
        # ===== 2. 원본 가격 데이터 병합 =====
        market_df = market_df.merge(
            self.raw_prices, 
            on='date', 
            how='left'
        )
        
        # ===== 3. 가격 정렬 검증 =====
        if 'next_open' in market_df.columns and 'ETH_Open' in market_df.columns:
            # next_open은 shift(-1)이므로, ETH_Open의 다음 날과 일치해야 함
            market_df['_check'] = market_df['next_open'] - market_df['ETH_Open'].shift(-1)
            misalignment = market_df['_check'].abs().max()
            
            if pd.notna(misalignment) and misalignment > 0.01:
                print(f"  ⚠️  Fold {fold}: 가격 정렬 불일치 (max diff: {misalignment:.2f})")
            
            market_df.drop('_check', axis=1, inplace=True)
        
        # ===== 4. NaN 체크 =====
        price_nan = market_df[['ETH_Open', 'ETH_Close', 'next_open', 'next_close']].isna().sum()
        if price_nan.any():
            print(f"  ⚠️  Fold {fold} NaN 발견: {price_nan[price_nan > 0].to_dict()}")
        
        # ===== 5. 모델 예측 로드 =====
        predictions = {}
        pred_base = self.config.base_path / "fold_results/direction" / fold_name
        
        for model in self.config.models:
            pred_path = pred_base / f"{model}_predictions.csv"
            if pred_path.exists():
                pred_df = pd.read_csv(pred_path)
                pred_df['date'] = pd.to_datetime(pred_df['date'])
                predictions[model] = pred_df
        
        return market_df, predictions
    
    def calculate_market_regime(self, market_df: pd.DataFrame) -> str:
        """시장 체제 분류"""
        if 'next_close' not in market_df.columns or 'next_open' not in market_df.columns:
            return 'UNKNOWN'
        
        total_return = (market_df['next_close'].iloc[-1] / market_df['next_open'].iloc[0] - 1) * 100
        
        if total_return >= 5:
            return 'BULL'
        elif total_return <= -5:
            return 'BEAR'
        else:
            return 'SIDEWAYS'

# ==================== 포지션 관리 ====================
class PositionManager:
    def __init__(self, config: Config):
        self.config = config
        self.trade_history = []
    
    def calculate_position_size(self, strategy: str, capital: float, 
                               confidence: float = 0, trades: List = None) -> float:
        """포지션 크기 계산"""
        if strategy.startswith('fixed_'):
            ratio = self.config.position_strategies[strategy]
            return capital * ratio
        
        elif strategy == 'confidence_based':
            return capital * confidence * 0.3
        
        elif strategy == 'kelly':
            if trades is None or len(trades) < self.config.kelly_min_trades:
                return capital * 0.10  # 기본값
            
            wins = [t for t in trades if t['pnl'] > 0]
            losses = [t for t in trades if t['pnl'] < 0]
            
            if not wins or not losses:
                return capital * 0.10
            
            win_rate = len(wins) / len(trades)
            avg_win = np.mean([t['pnl_pct'] for t in wins])
            avg_loss = abs(np.mean([t['pnl_pct'] for t in losses]))
            
            kelly_pct = (win_rate * avg_win - (1 - win_rate) * avg_loss) / avg_win
            kelly_pct = np.clip(kelly_pct, 0.01, 0.50)  # 1-50% 제한
            
            return capital * kelly_pct
        
        return capital * 0.10

# ==================== Day Trading 백테스터 ====================
class DayTradingBacktester:
    def __init__(self, config: Config):
        self.config = config
        self.position_manager = PositionManager(config)
    
    def backtest(self, market_df: pd.DataFrame, pred_df: pd.DataFrame, 
                 strategy: str) -> Dict:
        """
        Day Trading 백테스팅
        
        market_df 구조:
        - date: 예측일 (T)
        - ETH_Open, ETH_Close: T일 실제 가격 (현재가)
        - next_open: T+1일 시가 (진입)
        - next_close: T+1일 종가 (청산)
        """
        capital = self.config.initial_capital
        trades = []
        daily_capital = []
        
        # 병합
        merged_df = pred_df.merge(market_df, on='date', how='left')
        
        for idx, row in merged_df.iterrows():
            pred_direction = row['pred_direction']
            confidence = row.get('confidence', 0.5)
            
            # ===== 가격 검증 =====
            if pd.isna(row.get('next_open')) or pd.isna(row.get('next_close')):
                daily_capital.append(capital)
                continue
            
            if pred_direction == 1:  # 매수 신호
                position_size = self.position_manager.calculate_position_size(
                    strategy, capital, confidence, trades
                )
                
                # T일 종가에 예측 → T+1일 시가에 진입
                entry_price = row['next_open']
                exit_price = row['next_close']
                
                # 거래 비용
                trading_cost = position_size * self.config.trading_cost * 2
                
                # 손익
                gross_pnl = position_size * (exit_price / entry_price - 1)
                net_pnl = gross_pnl - trading_cost
                
                capital += net_pnl
                
                trades.append({
                    'date': row['date'],
                    'type': 'day_trade',
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'position_size': position_size,
                    'pnl': net_pnl,
                    'pnl_pct': net_pnl / position_size,
                    'confidence': confidence
                })
            
            daily_capital.append(capital)
        
        return self._calculate_metrics(trades, daily_capital, market_df)
    
    def _calculate_metrics(self, trades: List, daily_capital: List, 
                          market_df: pd.DataFrame) -> Dict:
        """성과 지표 계산"""
        if not trades:
            return self._empty_metrics()
        
        final_capital = daily_capital[-1]
        total_return = (final_capital / self.config.initial_capital - 1) * 100
        
        # 거래 분석
        winning_trades = [t for t in trades if t['pnl'] > 0]
        losing_trades = [t for t in trades if t['pnl'] < 0]
        
        win_rate = len(winning_trades) / len(trades) if trades else 0
        
        total_wins = sum(t['pnl'] for t in winning_trades)
        total_losses = abs(sum(t['pnl'] for t in losing_trades))
        profit_factor = total_wins / total_losses if total_losses > 0 else 0
        
        # 최대 낙폭
        peak = self.config.initial_capital
        max_dd = 0
        for capital in daily_capital:
            if capital > peak:
                peak = capital
            dd = (peak - capital) / peak
            if dd > max_dd:
                max_dd = dd
        
        # Sharpe/Sortino
        returns = pd.Series(daily_capital).pct_change().dropna()
        sharpe = returns.mean() / returns.std() * np.sqrt(252) if len(returns) > 0 and returns.std() > 0 else 0
        
        negative_returns = returns[returns < 0]
        sortino = returns.mean() / negative_returns.std() * np.sqrt(252) if len(negative_returns) > 0 and negative_returns.std() > 0 else 0
        
        # 연간 수익률
        days = len(daily_capital)
        annual_return = ((final_capital / self.config.initial_capital) ** (252 / days) - 1) * 100 if days > 0 else 0
        
        return {
            'total_return_pct': total_return,
            'annual_return_pct': annual_return,
            'num_trades': len(trades),
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'max_drawdown': max_dd,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino,
            'final_capital': final_capital,
            'trades': trades
        }
    
    def _empty_metrics(self) -> Dict:
        return {
            'total_return_pct': 0, 'annual_return_pct': 0,
            'num_trades': 0, 'win_rate': 0, 'profit_factor': 0,
            'max_drawdown': 0, 'sharpe_ratio': 0, 'sortino_ratio': 0,
            'final_capital': self.config.initial_capital, 'trades': []
        }

# ==================== Swing Trading 백테스터 ====================
class SwingTradingBacktester:
    def __init__(self, config: Config):
        self.config = config
        self.position_manager = PositionManager(config)
    
    def backtest(self, market_df: pd.DataFrame, pred_df: pd.DataFrame, 
                 strategy: str) -> Dict:
        """
        Swing Trading 백테스팅
        
        진입: T+1일 시가 (next_open)
        청산: 조건 충족 시 당일 종가 (ETH_Close)
        """
        capital = self.config.initial_capital
        trades = []
        daily_capital = []
        
        in_position = False
        entry_price = 0
        entry_date = None
        position_size = 0
        holding_days = 0
        entry_cost = 0
        
        merged_df = pred_df.merge(market_df, on='date', how='left')
        
        for idx in range(len(merged_df)):
            row = merged_df.iloc[idx]
            
            if not in_position:
                # ===== 진입 조건 =====
                trend_strength = self._calculate_trend_strength(merged_df, idx)
                
                if (trend_strength > self.config.swing_entry_threshold and
                    row['pred_direction'] == 1 and
                    row.get('confidence', 0) > self.config.swing_min_confidence and
                    pd.notna(row.get('next_open'))):
                    
                    position_size = self.position_manager.calculate_position_size(
                        strategy, capital, row.get('confidence', 0.5), trades
                    )
                    
                    # 진입: T+1일 시가
                    entry_price = row['next_open']
                    entry_date = row['date']
                    entry_cost = position_size * self.config.trading_cost
                    
                    in_position = True
                    holding_days = 0
            
            else:
                # ===== 청산 조건 =====
                holding_days += 1
                
                # ✅ 현재가: 당일 종가 (ETH_Close)
                current_price = row.get('ETH_Close', entry_price)
                if pd.isna(current_price):
                    current_price = entry_price
                
                current_return = (current_price / entry_price - 1)
                trend_strength = self._calculate_trend_strength(merged_df, idx)
                
                should_exit = (
                    current_return <= self.config.swing_stop_loss or
                    current_return >= self.config.swing_take_profit or
                    trend_strength < self.config.swing_exit_threshold or
                    row['pred_direction'] == 0 or
                    holding_days >= self.config.swing_max_holding or
                    idx == len(merged_df) - 1
                )
                
                if should_exit:
                    exit_price = current_price
                    exit_cost = position_size * self.config.trading_cost
                    
                    gross_pnl = position_size * (exit_price / entry_price - 1)
                    net_pnl = gross_pnl - entry_cost - exit_cost
                    
                    capital += net_pnl
                    
                    trades.append({
                        'entry_date': entry_date,
                        'exit_date': row['date'],
                        'type': 'swing_trade',
                        'entry_price': entry_price,
                        'exit_price': exit_price,
                        'position_size': position_size,
                        'holding_days': holding_days,
                        'pnl': net_pnl,
                        'pnl_pct': net_pnl / position_size,
                        'exit_reason': self._get_exit_reason(
                            current_return, trend_strength, 
                            row['pred_direction'], holding_days, 
                            idx, len(merged_df)
                        )
                    })
                    
                    in_position = False
            
            daily_capital.append(capital)
        
        return self._calculate_metrics(trades, daily_capital, market_df)
    
    def _calculate_trend_strength(self, df: pd.DataFrame, idx: int) -> float:
        """최근 N일 트렌드 강도 계산"""
        start_idx = max(0, idx - self.config.swing_trend_window + 1)
        window_df = df.iloc[start_idx:idx+1]
        
        if len(window_df) == 0:
            return 0
        
        up_ratio = window_df['pred_direction'].sum() / len(window_df)
        avg_confidence = window_df.get('confidence', pd.Series([0.5]*len(window_df))).mean()
        
        return up_ratio * avg_confidence
    
    def _get_exit_reason(self, return_pct: float, trend: float, 
                        pred: int, days: int, idx: int, total_len: int) -> str:
        """청산 사유"""
        if idx == total_len - 1:
            return 'end_of_period'
        if return_pct <= self.config.swing_stop_loss:
            return 'stop_loss'
        elif return_pct >= self.config.swing_take_profit:
            return 'take_profit'
        elif trend < self.config.swing_exit_threshold:
            return 'trend_weakening'
        elif pred == 0:
            return 'reversal_signal'
        elif days >= self.config.swing_max_holding:
            return 'max_holding'
        return 'unknown'
    
    def _calculate_metrics(self, trades: List, daily_capital: List, 
                          market_df: pd.DataFrame) -> Dict:
        """성과 지표"""
        backtester = DayTradingBacktester(self.config)
        return backtester._calculate_metrics(trades, daily_capital, market_df)

# ==================== Buy & Hold 백테스터 ====================
class BuyHoldBacktester:
    def __init__(self, config: Config):
        self.config = config
    
    def backtest(self, market_df: pd.DataFrame) -> Dict:
        """Buy & Hold 백테스팅"""
        if 'next_open' not in market_df.columns or 'next_close' not in market_df.columns:
            return {
                'total_return_pct': 0, 'annual_return_pct': 0,
                'num_trades': 0, 'win_rate': 0, 'profit_factor': 0,
                'max_drawdown': 0, 'sharpe_ratio': 0, 'sortino_ratio': 0,
                'final_capital': self.config.initial_capital, 'trades': []
            }
        
        entry_price = market_df['next_open'].iloc[0]
        exit_price = market_df['next_close'].iloc[-1]
        
        capital = self.config.initial_capital
        position_size = capital
        
        # 매수/매도 비용
        entry_cost = position_size * self.config.trading_cost
        exit_cost = position_size * self.config.trading_cost
        
        gross_pnl = position_size * (exit_price / entry_price - 1)
        net_pnl = gross_pnl - entry_cost - exit_cost
        
        final_capital = capital + net_pnl
        total_return = (final_capital / capital - 1) * 100
        
        days = len(market_df)
        annual_return = ((final_capital / capital) ** (252 / days) - 1) * 100 if days > 0 else 0
        
        return {
            'total_return_pct': total_return,
            'annual_return_pct': annual_return,
            'num_trades': 1,
            'win_rate': 1 if net_pnl > 0 else 0,
            'profit_factor': 0,
            'max_drawdown': 0,
            'sharpe_ratio': 0,
            'sortino_ratio': 0,
            'final_capital': final_capital,
            'trades': [{
                'entry_date': market_df['date'].iloc[0],
                'exit_date': market_df['date'].iloc[-1],
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': net_pnl
            }]
        }

# ==================== 메인 백테스팅 엔진 ====================
class BacktestEngine:
    def __init__(self, config: Config):
        self.config = config
        self.loader = ImprovedDataLoader(config)  # ✅ 개선된 로더 사용
        self.day_backtester = DayTradingBacktester(config)
        self.swing_backtester = SwingTradingBacktester(config)
        self.buyhold_backtester = BuyHoldBacktester(config)
        
        self.results = defaultdict(list)
        self.buyhold_results = {}
        self.market_regimes = {}
    
    def run_all_backtests(self):
        """전체 백테스팅 실행"""
        print("=" * 80)
        print("이더리움 트레이딩 백테스팅 시스템 (Data Leakage 방지)")
        print("=" * 80)
        
        total_backtests = len(self.config.folds) * len(self.config.models) * \
                         len(self.config.position_strategies) * 2  # Day + Swing
        
        print(f"\n처리 중: {len(self.config.folds)} Folds × {len(self.config.models)} Models × "
              f"{len(self.config.position_strategies)} Strategies × 2 Methods = {total_backtests} backtests\n")
        
        # Fold별 처리
        for fold in self.config.folds:
            print(f"Processing Fold {fold}...")
            
            try:
                market_df, predictions = self.loader.load_fold_data(fold)
                
                # 시장 체제 분류
                regime = self.loader.calculate_market_regime(market_df)
                self.market_regimes[fold] = regime
                
                # Buy & Hold 백테스팅
                buyhold_result = self.buyhold_backtester.backtest(market_df)
                self.buyhold_results[fold] = buyhold_result
                
                # 모델별 백테스팅
                for model in self.config.models:
                    if model not in predictions:
                        continue
                    
                    pred_df = predictions[model]
                    
                    for strategy in self.config.position_strategies.keys():
                        # Day Trading
                        try:
                            day_result = self.day_backtester.backtest(market_df, pred_df, strategy)
                            self._store_result('day_trading', fold, model, strategy, 
                                              day_result, buyhold_result, regime)
                        except Exception as e:
                            print(f"  ⚠️  {model} Day Trading 오류: {str(e)[:50]}")
                        
                        # Swing Trading
                        try:
                            swing_result = self.swing_backtester.backtest(market_df, pred_df, strategy)
                            self._store_result('swing_trading', fold, model, strategy, 
                                              swing_result, buyhold_result, regime)
                        except Exception as e:
                            print(f"  ⚠️  {model} Swing Trading 오류: {str(e)[:50]}")
            
            except Exception as e:
                print(f"  ❌ Fold {fold} 처리 실패: {str(e)}")
                continue
        
        self._save_results()
        self._print_summary()
    
    def _store_result(self, method: str, fold: int, model: str, strategy: str,
                     result: Dict, buyhold_result: Dict, regime: str):
        """결과 저장"""
        alpha = result['total_return_pct'] - buyhold_result['total_return_pct']
        
        self.results[method].append({
            'fold': fold,
            'model': model,
            'strategy': strategy,
            'regime': regime,
            'total_return_pct': result['total_return_pct'],
            'annual_return_pct': result['annual_return_pct'],
            'alpha': alpha,
            'num_trades': result['num_trades'],
            'win_rate': result['win_rate'],
            'profit_factor': result['profit_factor'],
            'max_drawdown': result['max_drawdown'],
            'sharpe_ratio': result['sharpe_ratio'],
            'sortino_ratio': result['sortino_ratio'],
            'final_capital': result['final_capital'],
            'buyhold_return': buyhold_result['total_return_pct']
        })
    
    def _save_results(self):
        """결과 파일 저장"""
        print("\n결과 저장 중...")
        
        # 1. Day Trading Results
        if self.results['day_trading']:
            day_df = pd.DataFrame(self.results['day_trading'])
            day_df.to_csv(self.config.output_path / "day_trading_results.csv", index=False)
        
        # 2. Swing Trading Results
        if self.results['swing_trading']:
            swing_df = pd.DataFrame(self.results['swing_trading'])
            swing_df.to_csv(self.config.output_path / "swing_trading_results.csv", index=False)
        
        # 3. Buy & Hold Benchmark
        if self.buyhold_results:
            buyhold_list = []
            for fold, result in self.buyhold_results.items():
                buyhold_list.append({
                    'fold': fold,
                    'regime': self.market_regimes.get(fold, 'UNKNOWN'),
                    'total_return_pct': result['total_return_pct'],
                    'annual_return_pct': result['annual_return_pct']
                })
            buyhold_df = pd.DataFrame(buyhold_list)
            buyhold_df.to_csv(self.config.output_path / "buy_hold_benchmark.csv", index=False)
        
        # 4. Comparison Summary
        if self.results['day_trading'] or self.results['swing_trading']:
            summary_list = []
            
            day_df = pd.DataFrame(self.results['day_trading']) if self.results['day_trading'] else pd.DataFrame()
            swing_df = pd.DataFrame(self.results['swing_trading']) if self.results['swing_trading'] else pd.DataFrame()
            
            if not day_df.empty:
                day_df['method'] = 'day_trading'
            if not swing_df.empty:
                swing_df['method'] = 'swing_trading'
            
            all_df = pd.concat([day_df, swing_df], ignore_index=True)
            
            if not all_df.empty:
                for (model, strategy, method), group in all_df.groupby(['model', 'strategy', 'method']):
                    bull_alpha = group[group['regime'] == 'BULL']['alpha'].mean() if 'BULL' in group['regime'].values else 0
                    bear_alpha = group[group['regime'] == 'BEAR']['alpha'].mean() if 'BEAR' in group['regime'].values else 0
                    profitable_folds = (group['alpha'] > 0).sum()
                    
                    summary_list.append({
                        'model': model,
                        'strategy': strategy,
                        'method': method,
                        'avg_alpha': group['alpha'].mean(),
                        'avg_return': group['total_return_pct'].mean(),
                        'bull_alpha': bull_alpha,
                        'bear_alpha': bear_alpha,
                        'profitable_folds': profitable_folds,
                        'avg_win_rate': group['win_rate'].mean(),
                        'avg_sharpe': group['sharpe_ratio'].mean()
                    })
                
                summary_df = pd.DataFrame(summary_list)
                summary_df.to_csv(self.config.output_path / "comparison_summary.csv", index=False)
                
                # 5. Top 20 Models
                top20 = summary_df.nlargest(20, 'avg_alpha')
                top20.to_csv(self.config.output_path / "top_20_models.csv", index=False)
        
        print(f"✓ 결과 저장 완료: {self.config.output_path}")
    
    def _print_summary(self):
        """요약 출력"""
        print("\n" + "=" * 80)
        print("백테스팅 완료 - 요약")
        print("=" * 80)
        
        # 시장 체제
        if self.market_regimes:
            regimes = pd.Series(self.market_regimes.values()).value_counts()
            print(f"\n시장 체제: BULL {regimes.get('BULL', 0)}개, BEAR {regimes.get('BEAR', 0)}개, "
                  f"SIDEWAYS {regimes.get('SIDEWAYS', 0)}개")
        
        # TOP 20
        summary_path = self.config.output_path / "comparison_summary.csv"
        if summary_path.exists():
            summary_df = pd.read_csv(summary_path)
            top20 = summary_df.nlargest(20, 'avg_alpha')
            
            print("\n" + "-" * 80)
            print("TOP 20 MODELS BY ALPHA")
            print("-" * 80)
            
            for idx, row in top20.head(10).iterrows():
                print(f"[Rank {idx+1}] {row['model']} + {row['strategy']} + {row['method']}")
                print(f"  Avg Alpha: {row['avg_alpha']:+.1f}%, Profitable: {int(row['profitable_folds'])}/7, "
                      f"Bull Alpha: {row['bull_alpha']:+.1f}%, Bear Alpha: {row['bear_alpha']:+.1f}%")
            
            # 방법론 비교
            print("\n" + "-" * 80)
            print("METHOD COMPARISON")
            print("-" * 80)
            
            day_results = summary_df[summary_df['method'] == 'day_trading']
            swing_results = summary_df[summary_df['method'] == 'swing_trading']
            
            if not day_results.empty and not swing_results.empty:
                day_avg_return = day_results['avg_return'].mean()
                day_avg_alpha = day_results['avg_alpha'].mean()
                swing_avg_return = swing_results['avg_return'].mean()
                swing_avg_alpha = swing_results['avg_alpha'].mean()
                
                if self.buyhold_results:
                    buyhold_avg = pd.DataFrame([r for r in self.buyhold_results.values()])['total_return_pct'].mean()
                    print(f"Buy & Hold: {buyhold_avg:+.1f}% (baseline)")
                
                print(f"Day Trading: {day_avg_return:+.1f}% (Alpha {day_avg_alpha:+.1f}%)")
                print(f"Swing Trading: {swing_avg_return:+.1f}% (Alpha {swing_avg_alpha:+.1f}%)")
                
                best_method = 'Swing Trading' if swing_avg_alpha > day_avg_alpha else 'Day Trading'
                best_alpha = max(swing_avg_alpha, day_avg_alpha)
                print(f"\n→ {best_method}이 {best_alpha:+.1f}% 우수")
        
        print("\n" + "=" * 80)

# ==================== 강화학습 기반 동적 포지션 관리 ====================
class RLPositionManager:
    """Q-Learning 기반 포지션 크기 결정"""
    def __init__(self, config: Config):
        self.config = config
        self.q_table = {}
        self.epsilon = 0.1
        self.alpha = 0.1
        self.gamma = 0.95
        self.actions = [0.1, 0.3, 0.5, 0.7]
    
    def get_state(self, confidence: float, recent_performance: float, 
                  volatility: float) -> tuple:
        """상태 이산화"""
        conf_bin = int(confidence * 10) / 10
        perf_bin = 'good' if recent_performance > 0.02 else 'bad' if recent_performance < -0.02 else 'neutral'
        vol_bin = 'high' if volatility > 0.03 else 'low'
        return (conf_bin, perf_bin, vol_bin)
    
    def select_action(self, state: tuple, training: bool = True) -> float:
        """행동 선택 (ε-greedy)"""
        if state not in self.q_table:
            self.q_table[state] = {a: 0.0 for a in self.actions}
        
        if training and np.random.random() < self.epsilon:
            return np.random.choice(self.actions)
        
        return max(self.q_table[state].items(), key=lambda x: x[1])[0]
    
    def update_q_value(self, state: tuple, action: float, reward: float, 
                       next_state: tuple):
        """Q-테이블 업데이트"""
        if state not in self.q_table:
            self.q_table[state] = {a: 0.0 for a in self.actions}
        if next_state not in self.q_table:
            self.q_table[next_state] = {a: 0.0 for a in self.actions}
        
        current_q = self.q_table[state][action]
        max_next_q = max(self.q_table[next_state].values())
        
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.q_table[state][action] = new_q

# ==================== 동적 포지션 크기 조정 ====================
class DynamicPositionManager(PositionManager):
    """변동성과 추세 기반 동적 포지션 조정"""
    
    def calculate_dynamic_position(self, capital: float, confidence: float,
                                   recent_volatility: float, trend_strength: float,
                                   win_streak: int) -> float:
        """다중 요소 기반 포지션 크기"""
        base_size = 0.3
        
        confidence_factor = confidence * 0.5 + 0.5
        volatility_factor = max(0.5, 1.0 - recent_volatility * 10)
        trend_factor = 0.8 + trend_strength * 0.4
        streak_bonus = 1.0 + min(win_streak, 3) * 0.05
        
        final_size = base_size * confidence_factor * volatility_factor * trend_factor * streak_bonus
        final_size = np.clip(final_size, 0.05, 0.7)
        
        return capital * final_size

# ==================== 확장된 백테스팅 엔진 ====================
class EnhancedBacktestEngine(BacktestEngine):
    """강화학습 및 LLM 통합 백테스팅"""
    
    def __init__(self, config: Config):
        super().__init__(config)
        self.rl_manager = RLPositionManager(config)
        self.dynamic_manager = DynamicPositionManager(config)
    
    def run_enhanced_backtests(self):
        """강화학습/동적 포지션 백테스팅"""
        print("\n" + "=" * 80)
        print("고급 전략 백테스팅 (RL + Dynamic)")
        print("=" * 80)
        
        enhanced_results = []
        
        for fold in self.config.folds:
            try:
                market_df, predictions = self.loader.load_fold_data(fold)
                regime = self.market_regimes.get(fold, self.loader.calculate_market_regime(market_df))
                
                for model in self.config.models:
                    if model not in predictions:
                        continue
                    
                    pred_df = predictions[model]
                    
                    # RL 기반
                    rl_result = self._backtest_with_rl(market_df, pred_df, fold)
                    enhanced_results.append({
                        'fold': fold, 'model': model, 'strategy': 'rl_position',
                        'method': 'swing_trading', 'regime': regime, **rl_result
                    })
                    
                    # 동적 포지션
                    dynamic_result = self._backtest_with_dynamic(market_df, pred_df, fold)
                    enhanced_results.append({
                        'fold': fold, 'model': model, 'strategy': 'dynamic_position',
                        'method': 'day_trading', 'regime': regime, **dynamic_result
                    })
            
            except Exception as e:
                print(f"  ⚠️  Fold {fold} 고급 전략 오류: {str(e)[:50]}")
                continue
        
        if enhanced_results:
            enhanced_df = pd.DataFrame(enhanced_results)
            enhanced_df.to_csv(self.config.output_path / "enhanced_strategies.csv", index=False)
            print(f"✓ 고급 전략 백테스팅 완료: {len(enhanced_results)} 결과")
    
    def _backtest_with_rl(self, market_df: pd.DataFrame, pred_df: pd.DataFrame,
                         fold: int) -> dict:
        """강화학습 기반 포지션 관리"""
        capital = self.config.initial_capital
        trades = []
        recent_returns = []
        
        merged_df = pred_df.merge(market_df, on='date')
        
        for idx, row in merged_df.iterrows():
            if row['pred_direction'] == 1 and pd.notna(row.get('next_open')) and pd.notna(row.get('next_close')):
                recent_perf = np.mean(recent_returns[-10:]) if recent_returns else 0
                volatility = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
                
                state = self.rl_manager.get_state(row.get('confidence', 0.5), recent_perf, volatility)
                action = self.rl_manager.select_action(state, training=True)
                
                position_size = capital * action
                
                entry_price = row['next_open']
                exit_price = row['next_close']
                
                pnl = position_size * (exit_price / entry_price - 1) - \
                      position_size * self.config.trading_cost * 2
                
                capital += pnl
                return_pct = pnl / position_size
                recent_returns.append(return_pct)
                
                reward = return_pct * 100
                next_state = self.rl_manager.get_state(row.get('confidence', 0.5), return_pct, volatility)
                self.rl_manager.update_q_value(state, action, reward, next_state)
                
                trades.append({'pnl': pnl, 'pnl_pct': return_pct})
        
        return self._calculate_simple_metrics(trades, capital)
    
    def _backtest_with_dynamic(self, market_df: pd.DataFrame, 
                              pred_df: pd.DataFrame, fold: int) -> dict:
        """동적 포지션 관리"""
        capital = self.config.initial_capital
        trades = []
        win_streak = 0
        recent_returns = []
        
        merged_df = pred_df.merge(market_df, on='date')
        
        for idx in range(len(merged_df)):
            row = merged_df.iloc[idx]
            
            if row['pred_direction'] == 1 and pd.notna(row.get('next_open')) and pd.notna(row.get('next_close')):
                volatility = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
                
                start_idx = max(0, idx - 5)
                trend_strength = merged_df.iloc[start_idx:idx+1]['pred_direction'].mean()
                
                position_size = self.dynamic_manager.calculate_dynamic_position(
                    capital, row.get('confidence', 0.5), volatility, trend_strength, win_streak
                )
                
                entry_price = row['next_open']
                exit_price = row['next_close']
                
                pnl = position_size * (exit_price / entry_price - 1) - \
                      position_size * self.config.trading_cost * 2
                
                capital += pnl
                return_pct = pnl / position_size
                recent_returns.append(return_pct)
                
                if pnl > 0:
                    win_streak += 1
                else:
                    win_streak = 0
                
                trades.append({'pnl': pnl, 'pnl_pct': return_pct})
        
        return self._calculate_simple_metrics(trades, capital)
    
    def _calculate_simple_metrics(self, trades: List, final_capital: float) -> dict:
        """간단한 메트릭"""
        if not trades:
            return {'total_return_pct': 0, 'alpha': 0, 'num_trades': 0, 'win_rate': 0}
        
        total_return = (final_capital / self.config.initial_capital - 1) * 100
        wins = [t for t in trades if t['pnl'] > 0]
        
        return {
            'total_return_pct': total_return,
            'alpha': total_return,
            'num_trades': len(trades),
            'win_rate': len(wins) / len(trades),
            'final_capital': final_capital
        }

# ==================== 실행 ====================
if __name__ == "__main__":
    config = Config()
    
    # ===== 기본 백테스팅 =====
    print("\n🚀 기본 백테스팅 시작")
    engine = BacktestEngine(config)
    engine.run_all_backtests()
    
    # ===== 고급 백테스팅 (옵션) =====
    print("\n🚀 고급 백테스팅 시작")
    enhanced_engine = EnhancedBacktestEngine(config)
    enhanced_engine.run_enhanced_backtests()
    
    print("\n✓ 모든 백테스팅 완료!")
    print(f"✓ 결과 파일 위치: {config.output_path}")



🚀 기본 백테스팅 시작
Loading raw price data from: ../macro_data/macro_data/macro_crypto_data.csv
✓ Loaded 3219 rows of price data
이더리움 트레이딩 백테스팅 시스템 (Data Leakage 방지)

처리 중: 7 Folds × 29 Models × 8 Strategies × 2 Methods = 3248 backtests

Processing Fold 1...
  ⚠️  SVM Day Trading 오류: float division by zero
Processing Fold 2...
  ⚠️  SVM Day Trading 오류: float division by zero
Processing Fold 3...
  ⚠️  SVM Day Trading 오류: float division by zero
Processing Fold 4...


KeyboardInterrupt: 

In [4]:
"""
데이터 검증 스크립트 - 백테스팅 전 데이터 이상 탐지
"""

import pandas as pd
import numpy as np
from pathlib import Path

# 설정
BASE_PATH = Path("../model_results/2025-10-26")
RAW_DATA_PATH = Path("../macro_data/macro_data/macro_crypto_data.csv")
FOLDS = list(range(1, 8))
MODELS_TO_CHECK = ['SVM', 'RandomForest', 'LightGBM', 'XGBoost']  

def load_raw_prices():
    """원본 가격 데이터 로드"""
    print("Loading raw price data...")
    raw_prices = pd.read_csv(RAW_DATA_PATH)
    raw_prices['date'] = pd.to_datetime(raw_prices['date'])
    
    price_cols = ['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close', 'ETH_Volume']
    available_cols = [col for col in price_cols if col in raw_prices.columns]
    raw_prices = raw_prices[available_cols].copy()
    
    print(f"Loaded {len(raw_prices)} rows")
    return raw_prices

def load_fold_data(fold, raw_prices):
    """Fold별 데이터 로드"""
    fold_name = f"fold_{fold}_walk_forward_rolling_reverse"
    
    # 시장 데이터
    market_path = BASE_PATH / "raw_data/direction/walk_forward" / fold_name / "test_raw.csv"
    
    if not market_path.exists():
        print(f"Market data not found: {market_path}")
        return None, {}
    
    market_df = pd.read_csv(market_path)
    market_df['date'] = pd.to_datetime(market_df['date'])
    
    # Target 컬럼
    target_cols = ['date', 'next_open', 'next_close', 'next_direction', 'next_log_return']
    available_targets = [col for col in target_cols if col in market_df.columns]
    market_df = market_df[available_targets].copy()
    
    # 원본 가격 병합
    market_df = market_df.merge(raw_prices, on='date', how='left')
    
    # 모델 예측 로드
    predictions = {}
    pred_base = BASE_PATH / "fold_results/direction" / fold_name
    
    for model in MODELS_TO_CHECK:
        pred_path = pred_base / f"{model}_predictions.csv"
        if pred_path.exists():
            pred_df = pd.read_csv(pred_path)
            pred_df['date'] = pd.to_datetime(pred_df['date'])
            predictions[model] = pred_df
    
    return market_df, predictions

def validate_data(market_df, pred_df, model_name, fold):
    """데이터 검증"""
    print(f"\n{'='*80}")
    print(f"Fold {fold} - {model_name} 데이터 검증")
    print(f"{'='*80}")
    
    if market_df is None or market_df.empty:
        print("ERROR: Market data is empty or None")
        return
    
    if pred_df is None or pred_df.empty:
        print("ERROR: Prediction data is empty or None")
        return
    
    print(f"\n[데이터 크기]")
    print(f"Market rows: {len(market_df)}")
    print(f"Pred rows: {len(pred_df)}")
    
    # 날짜 범위
    print(f"\n[날짜 범위]")
    print(f"Market: {market_df['date'].min()} ~ {market_df['date'].max()}")
    print(f"Pred: {pred_df['date'].min()} ~ {pred_df['date'].max()}")
    
    # 가격 통계
    print(f"\n[가격 통계]")
    price_cols = ['next_open', 'next_close', 'ETH_Open', 'ETH_Close']
    available_price_cols = [col for col in price_cols if col in market_df.columns]
    
    if available_price_cols:
        print(market_df[available_price_cols].describe())
    else:
        print("WARNING: No price columns found")
    
    # 예측 분포
    print(f"\n[예측 분포]")
    if 'pred_direction' in pred_df.columns:
        print(pred_df['pred_direction'].value_counts())
        print(f"Upward predictions: {(pred_df['pred_direction'] == 1).sum()}")
        print(f"Downward predictions: {(pred_df['pred_direction'] == 0).sum()}")
    else:
        print("WARNING: pred_direction column not found")
    
    # Confidence 통계
    if 'confidence' in pred_df.columns:
        print(f"\n[Confidence 통계]")
        print(pred_df['confidence'].describe())
    
    # NaN 개수
    print(f"\n[NaN 개수]")
    for col in available_price_cols:
        nan_count = market_df[col].isna().sum()
        print(f"{col}: {nan_count}")
    
    # 0 이하 가격
    print(f"\n[이상 가격 (0 이하)]")
    for col in available_price_cols:
        zero_or_below = (market_df[col] <= 0).sum()
        if zero_or_below > 0:
            print(f"WARNING: {col} has {zero_or_below} values <= 0")
            print(f"  Min value: {market_df[col].min()}")
        else:
            print(f"{col}: OK")
    
    # 극단값
    print(f"\n[극단 가격 변화]")
    if 'next_open' in market_df.columns and 'next_close' in market_df.columns:
        market_df_clean = market_df[['next_open', 'next_close']].dropna()
        if len(market_df_clean) > 0:
            returns = (market_df_clean['next_close'] / market_df_clean['next_open'] - 1)
            extreme_returns = returns[abs(returns) > 0.5]
            
            if len(extreme_returns) > 0:
                print(f"WARNING: {len(extreme_returns)} extreme returns (>50%)")
                print(f"  Max return: {returns.max():.2%}")
                print(f"  Min return: {returns.min():.2%}")
            else:
                print("OK: No extreme returns")
    
    # 병합 후 데이터 손실 체크
    print(f"\n[병합 검증]")
    merged = pred_df.merge(market_df, on='date', how='left')
    merged_valid = merged[['next_open', 'next_close']].dropna()
    
    print(f"Original pred rows: {len(pred_df)}")
    print(f"After merge: {len(merged)}")
    print(f"Valid (no NaN): {len(merged_valid)}")
    print(f"Data loss: {len(pred_df) - len(merged_valid)} rows ({(len(pred_df) - len(merged_valid))/len(pred_df)*100:.1f}%)")

def main():
    """메인 실행"""
    print("="*80)
    print("백테스팅 데이터 검증 스크립트")
    print("="*80)
    
    # 원본 가격 로드
    raw_prices = load_raw_prices()
    
    # Fold별 검증
    for fold in FOLDS:
        print(f"\n\nProcessing Fold {fold}...")
        
        market_df, predictions = load_fold_data(fold, raw_prices)
        
        if market_df is None:
            print(f"Skipping Fold {fold}: Data loading failed")
            continue
        
        # 각 모델별 검증
        for model_name in MODELS_TO_CHECK:
            if model_name in predictions:
                pred_df = predictions[model_name]
                validate_data(market_df, pred_df, model_name, fold)
            else:
                print(f"\nWARNING: {model_name} predictions not found for Fold {fold}")
    
    print("\n" + "="*80)
    print("검증 완료")
    print("="*80)

if __name__ == "__main__":
    main()


백테스팅 데이터 검증 스크립트
Loading raw price data...
Loaded 3219 rows


Processing Fold 1...

Fold 1 - SVM 데이터 검증

[데이터 크기]
Market rows: 150
Pred rows: 150

[날짜 범위]
Market: 2022-12-07 00:00:00 ~ 2023-05-05 00:00:00
Pred: 2022-12-07 00:00:00 ~ 2023-05-05 00:00:00

[가격 통계]
         next_open   next_close     ETH_Open    ETH_Close
count   150.000000   150.000000   150.000000   150.000000
mean   1604.183429  1608.541800  1599.357256  1604.089905
std     247.956502   247.375085   247.333680   248.098470
min    1167.882690  1167.609863  1167.882690  1167.609863
25%    1431.515656  1441.899292  1420.734833  1431.533752
50%    1633.073853  1637.197021  1630.098022  1632.986145
75%    1794.596893  1803.997589  1793.613190  1795.031555
max    2120.001221  2120.005859  2120.001221  2120.005859

[예측 분포]
1    136
0     14
Name: pred_direction, dtype: int64
Upward predictions: 136
Downward predictions: 14

[Confidence 통계]
count    150.000000
mean       0.171530
std        0.108964
min        0.000000
25%     


Fold 3 - SVM 데이터 검증

[데이터 크기]
Market rows: 150
Pred rows: 150

[날짜 범위]
Market: 2023-10-03 00:00:00 ~ 2024-02-29 00:00:00
Pred: 2023-10-03 00:00:00 ~ 2024-02-29 00:00:00

[가격 통계]
         next_open   next_close     ETH_Open    ETH_Close
count   150.000000   150.000000   150.000000   150.000000
mean   2207.666781  2219.380427  2196.472924  2207.524639
std     401.014524   410.803404   392.483401   401.029175
min    1539.432861  1539.612427  1539.432861  1539.612427
25%    1940.605164  1960.981384  1908.845581  1943.020477
50%    2233.593872  2236.552124  2232.305420  2232.611572
75%    2358.306824  2368.834412  2356.872253  2358.443848
max    3386.802734  3435.053955  3386.802734  3385.703857

[예측 분포]
0    91
1    59
Name: pred_direction, dtype: int64
Upward predictions: 59
Downward predictions: 91

[Confidence 통계]
count    150.000000
mean       0.090512
std        0.066779
min        0.000000
25%        0.032755
50%        0.081989
75%        0.130853
max        0.274394
Name: confiden

count    150.000000
mean       0.122158
std        0.088272
min        0.000300
25%        0.036740
50%        0.105022
75%        0.212012
max        0.264644
Name: confidence, dtype: float64

[NaN 개수]
next_open: 0
next_close: 0
ETH_Open: 0
ETH_Close: 0

[이상 가격 (0 이하)]
next_open: OK
next_close: OK
ETH_Open: OK
ETH_Close: OK

[극단 가격 변화]
OK: No extreme returns

[병합 검증]
Original pred rows: 150
After merge: 150
Valid (no NaN): 150
Data loss: 0 rows (0.0%)


Processing Fold 6...

Fold 6 - SVM 데이터 검증

[데이터 크기]
Market rows: 150
Pred rows: 150

[날짜 범위]
Market: 2024-12-26 00:00:00 ~ 2025-05-24 00:00:00
Pred: 2024-12-26 00:00:00 ~ 2025-05-24 00:00:00

[가격 통계]
         next_open   next_close     ETH_Open    ETH_Close
count   150.000000   150.000000   150.000000   150.000000
mean   2444.672059  2439.465636  2451.088153  2444.662048
std     630.206443   626.054112   635.962964   630.213524
min    1472.601440  1472.553101  1472.601440  1472.553101
25%    1840.060638  1840.093140  1840.060638  1840.

In [6]:
"""
next_open/next_close shift 검증 - Data Leakage 확인
"""

import pandas as pd
import numpy as np
from pathlib import Path

BASE_PATH = Path("../model_results/2025-10-26")
RAW_DATA_PATH = Path("../macro_data/macro_data/macro_crypto_data.csv")
FOLDS = [1, 2, 3]  # 일부만 확인

def load_raw_prices():
    """원본 가격 데이터 로드"""
    raw_prices = pd.read_csv(RAW_DATA_PATH)
    raw_prices['date'] = pd.to_datetime(raw_prices['date'])
    price_cols = ['date', 'ETH_Open', 'ETH_Close']
    return raw_prices[price_cols].copy()

def validate_shift_operation(fold, raw_prices):
    """shift(-1) 연산 검증"""
    fold_name = f"fold_{fold}_walk_forward_rolling_reverse"
    
    # test_raw.csv 로드
    market_path = BASE_PATH / "raw_data/direction/walk_forward" / fold_name / "test_raw.csv"
    
    if not market_path.exists():
        print(f"Fold {fold}: File not found")
        return
    
    market_df = pd.read_csv(market_path)
    market_df['date'] = pd.to_datetime(market_df['date'])
    
    # 원본 가격 병합
    merged_df = market_df.merge(raw_prices, on='date', how='left')
    
    print(f"\n{'='*80}")
    print(f"Fold {fold} - shift(-1) 검증")
    print(f"{'='*80}")
    
    print(f"\n[1단계] test_raw.csv에서 next_open/next_close 확인")
    print(f"  - next_open 컬럼 존재: {'next_open' in market_df.columns}")
    print(f"  - next_close 컬럼 존재: {'next_close' in market_df.columns}")
    
    if 'next_open' not in market_df.columns or 'next_close' not in market_df.columns:
        print("  ERROR: Target columns missing!")
        return
    
    print(f"\n[2단계] 원본 데이터 병합 확인")
    print(f"  - ETH_Open 컬럼 존재: {'ETH_Open' in merged_df.columns}")
    print(f"  - ETH_Close 컬럼 존재: {'ETH_Close' in merged_df.columns}")
    
    if 'ETH_Open' not in merged_df.columns or 'ETH_Close' not in merged_df.columns:
        print("  ERROR: Original price columns missing!")
        return
    
    print(f"\n[3단계] shift(-1) 검증: next_open == ETH_Open의 다음 행?")
    print("-" * 80)
    
    # 처음 10개만 샘플 출력
    sample_df = merged_df[['date', 'next_open', 'ETH_Open', 'next_close', 'ETH_Close']].head(10).copy()
    
    # 다음 날 ETH_Open 계산
    sample_df['next_day_ETH_Open'] = sample_df['ETH_Open'].shift(-1)
    sample_df['next_day_ETH_Close'] = sample_df['ETH_Close'].shift(-1)
    
    # 차이 계산
    sample_df['diff_open'] = sample_df['next_open'] - sample_df['next_day_ETH_Open']
    sample_df['diff_close'] = sample_df['next_close'] - sample_df['next_day_ETH_Close']
    
    print(sample_df.to_string(index=False))
    
    print(f"\n[4단계] 전체 데이터 차이 통계")
    merged_df['expected_next_open'] = merged_df['ETH_Open'].shift(-1)
    merged_df['expected_next_close'] = merged_df['ETH_Close'].shift(-1)
    
    merged_df['diff_open'] = merged_df['next_open'] - merged_df['expected_next_open']
    merged_df['diff_close'] = merged_df['next_close'] - merged_df['expected_next_close']
    
    # NaN 제외 (마지막 행은 shift(-1)로 NaN)
    diff_open_clean = merged_df['diff_open'].dropna()
    diff_close_clean = merged_df['diff_close'].dropna()
    
    print(f"\n차이 통계 (next_open - ETH_Open.shift(-1)):")
    print(f"  Count: {len(diff_open_clean)}")
    print(f"  Mean: {diff_open_clean.mean():.6f}")
    print(f"  Std: {diff_open_clean.std():.6f}")
    print(f"  Max abs diff: {diff_open_clean.abs().max():.6f}")
    
    print(f"\n차이 통계 (next_close - ETH_Close.shift(-1)):")
    print(f"  Count: {len(diff_close_clean)}")
    print(f"  Mean: {diff_close_clean.mean():.6f}")
    print(f"  Std: {diff_close_clean.std():.6f}")
    print(f"  Max abs diff: {diff_close_clean.abs().max():.6f}")
    
    print(f"\n[5단계] 최종 판정")
    
    # 허용 오차: 0.01 (1센트)
    tolerance = 0.01
    
    if diff_open_clean.abs().max() < tolerance and diff_close_clean.abs().max() < tolerance:
        print(f"  STATUS: PASS")
        print(f"  - next_open은 정확히 다음 날 ETH_Open입니다.")
        print(f"  - next_close는 정확히 다음 날 ETH_Close입니다.")
        print(f"  - shift(-1) 연산이 올바르게 적용되었습니다.")
    else:
        print(f"  STATUS: FAIL")
        print(f"  - 차이가 허용 범위({tolerance})를 초과합니다!")
        print(f"  - Data Leakage 또는 데이터 오정렬 가능성이 있습니다!")
        
        # 가장 큰 차이가 나는 행 출력
        max_diff_idx_open = diff_open_clean.abs().idxmax()
        max_diff_idx_close = diff_close_clean.abs().idxmax()
        
        print(f"\n  가장 큰 차이 (Open):")
        problem_row_open = merged_df.loc[max_diff_idx_open, ['date', 'next_open', 'expected_next_open', 'diff_open']]
        print(f"    {problem_row_open.to_dict()}")
        
        print(f"\n  가장 큰 차이 (Close):")
        problem_row_close = merged_df.loc[max_diff_idx_close, ['date', 'next_close', 'expected_next_close', 'diff_close']]
        print(f"    {problem_row_close.to_dict()}")
    
    print(f"\n[6단계] 백테스팅 시뮬레이션")
    print("-" * 80)
    
    # 실제 백테스팅 시나리오 재현
    test_idx = 5  # 6번째 날
    test_row = merged_df.iloc[test_idx]
    
    print(f"\n시나리오: {test_row['date']} 종가에 예측 수행")
    print(f"  - 예측 날짜: {test_row['date']}")
    print(f"  - 당일 종가: ${test_row['ETH_Close']:,.2f}")
    print(f"\n예측 결과: 상승 예측 (매수 신호)")
    print(f"  → 다음 날 ({merged_df.iloc[test_idx+1]['date']}) 시가에 진입")
    print(f"  → 다음 날 종가에 청산")
    
    print(f"\n백테스팅 사용 가격:")
    print(f"  - 진입가 (next_open): ${test_row['next_open']:,.2f}")
    print(f"  - 청산가 (next_close): ${test_row['next_close']:,.2f}")
    
    print(f"\n실제 다음 날 가격 (검증):")
    next_day = merged_df.iloc[test_idx + 1]
    print(f"  - 실제 다음 날 시가 (ETH_Open): ${next_day['ETH_Open']:,.2f}")
    print(f"  - 실제 다음 날 종가 (ETH_Close): ${next_day['ETH_Close']:,.2f}")
    
    print(f"\n일치 여부:")
    open_match = abs(test_row['next_open'] - next_day['ETH_Open']) < tolerance
    close_match = abs(test_row['next_close'] - next_day['ETH_Close']) < tolerance
    
    print(f"  - 진입가 일치: {'YES' if open_match else 'NO'} (차이: ${abs(test_row['next_open'] - next_day['ETH_Open']):.6f})")
    print(f"  - 청산가 일치: {'YES' if close_match else 'NO'} (차이: ${abs(test_row['next_close'] - next_day['ETH_Close']):.6f})")
    
    if open_match and close_match:
        print(f"\n  RESULT: 백테스팅 가격이 정확합니다. Data Leakage 없음.")
    else:
        print(f"\n  RESULT: 백테스팅 가격이 부정확합니다. 데이터 오류 가능성!")

def main():
    """메인 실행"""
    print("="*80)
    print("shift(-1) 연산 및 Data Leakage 검증")
    print("="*80)
    
    raw_prices = load_raw_prices()
    
    for fold in FOLDS:
        validate_shift_operation(fold, raw_prices)
    
    print("\n" + "="*80)
    print("검증 완료")
    print("="*80)

if __name__ == "__main__":
    main()


shift(-1) 연산 및 Data Leakage 검증

Fold 1 - shift(-1) 검증

[1단계] test_raw.csv에서 next_open/next_close 확인
  - next_open 컬럼 존재: True
  - next_close 컬럼 존재: True

[2단계] 원본 데이터 병합 확인
  - ETH_Open 컬럼 존재: True
  - ETH_Close 컬럼 존재: True

[3단계] shift(-1) 검증: next_open == ETH_Open의 다음 행?
--------------------------------------------------------------------------------
      date   next_open    ETH_Open  next_close   ETH_Close  next_day_ETH_Open  next_day_ETH_Close  diff_open  diff_close
2022-12-07 1232.451782 1271.553101 1281.116333 1232.437500        1232.451782         1281.116333        0.0         0.0
2022-12-08 1281.077271 1232.451782 1264.284790 1281.116333        1281.077271         1264.284790        0.0         0.0
2022-12-09 1264.375488 1281.077271 1266.384155 1264.284790        1264.375488         1266.384155        0.0         0.0
2022-12-10 1266.417847 1264.375488 1263.868530 1266.384155        1266.417847         1263.868530        0.0         0.0
2022-12-11 1263.573486 1266.417847 1274.